In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from os import listdir
from os.path import join
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
index_dir={'0':0, '1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'+':10,'-':11,'times':12}

In [ ]:
def get_index(directory):
    try:
        return index_dir[directory]
    except KeyError:
        print(f"Directory '{directory}' not found in index_dir.")
        return None

def load_images(folder):
    train_data = []

    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        #img = ~img
        if img is not None:
            img = ~img
            _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
            ctrs, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
            m = 0
            for c in cnt:
                x, y, w, h = cv2.boundingRect(c)
                m = max(w*h, m)
                if m == w*h:
                    x_max,y_max,w_max,h_max=x,y,w,h
            im_crop = img[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
            im_resize = cv2.resize(im_crop, (28, 28))
            im_resize = np.reshape(im_resize, (784, 1))
            train_data.append(im_resize)
    return train_data

In [ ]:
!unzip /content/drive/MyDrive/extracted_images-2.zip

In [ ]:
dataset_dir = '/content/extracted_images-2'
directory_list = listdir(dataset_dir)

first = True
data = []
print('Importing...')
for directory in directory_list:
    full_path = os.path.join(dataset_dir, directory)
    if os.path.isdir(full_path):
        print(directory)
        if first:
            first = False
            data = load_images(full_path)
            for i in range(len(data)):
                data[i] = np.append(data[i], [str(get_index(directory))])
            continue

        auxiliary_data = load_images(full_path)
        for i in range(len(auxiliary_data)):
            auxiliary_data[i] = np.append(auxiliary_data[i], [str(get_index(directory))])
        data = np.concatenate((data, auxiliary_data))

df = pd.DataFrame(data, index=None)
df.to_csv('./sample_data/train_data.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

df_train = pd.read_csv('/content/sample_data/train_data.csv', index_col=False)
labels = df_train[['784']]
df_train.head()

df_train.drop(df_train.columns[[784]], axis=1, inplace=True)
np.random.seed(1212)

labels = np.array(labels)

categorical_data = to_categorical(labels, num_classes=13)

l = []
for i in range(df_train.shape[0]):
    l.append(np.array(df_train[i:i + 1]).reshape(28, 28, 1))

print(len(l))

train_X, test_X, train_y, test_y = train_test_split(np.array(l), categorical_data, test_size=0.20, random_state=42)

print(len(train_X))
print(len(train_y))
print(len(test_X))

np.random.seed(7)

In [ ]:
from tensorflow import keras

model = keras.Sequential([
    layers.Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'),
    layers.MaxPool2D(pool_size=2),
    layers.Conv2D(15, (3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=2),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(13, activation='softmax'),
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_X, train_y, validation_split=0.25, epochs=10, batch_size=200, shuffle=True, verbose=1)

In [ ]:
model.evaluate(test_X, test_y)

In [ ]:
model.summary()

In [ ]:
model_json = model.to_json()
with open('/content/sample_data/model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_weights.h5')


In [ ]:
from tensorflow.keras.models import model_from_json

print('Loading Model...')
model_json = open('/content/sample_data/model.json', 'r')
loaded_model_json = model_json.read()
model_json.close()
model = model_from_json(loaded_model_json)

print('Loading weights...')
model.load_weights("/content/sample_data/model_weights.h5")

In [ ]:
labels = [ '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','+','-','x']

In [ ]:
from PIL import Image,ImageDraw,ImageGrab

In [ ]:
def activate_event(event):
        global lasx,lasy
        lasx,lasy=event.x,event.y

In [ ]:
def draw_smth(event):
    global lasx,lasy
    cv.create_line((lasx,lasy,event.x,event.y),fill='black',width=4)
    #cv.draw.line([x1, y1, x2, y2], fill="black", width=7)
    lasx,lasy=event.x,event.y

In [ ]:
def save():
    filename="/content/sample_data/Screenshot 2023-07-04 at 12.41.36 AM.png"
    widget = cv
    x=root.winfo_rootx()+widget.winfo_x()+50
    y=root.winfo_rooty()+widget.winfo_y()+50
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()

    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)

In [ ]:
from sympy import *

class Solver:
    def __init__(self, equation):
        self.equation = str(equation)
        self.leftEqu = []

    def convertEquationIntoGeneralForm(self):
        leftSide, rightSide = '', ''
        equalIndx = self.equation.index('=')
        leftSide = self.equation[0:equalIndx]
        rightSide = self.equation[equalIndx + 1:len(self.equation)]

        if rightSide[0].isalpha() or rightSide[0].isdigit():
            rightSide = '+' + rightSide

        for i in range(0, len(rightSide)):
            if rightSide[i] == '+':
                rightSide = rightSide[0:i] + '-' + rightSide[i + 1:len(rightSide)]
            elif rightSide[i] == '-':
                rightSide = rightSide[0:i] + '+' + rightSide[i + 1:len(rightSide)]
            leftSide += rightSide[i]

        self.equation = leftSide + '=' + '0'
        self.leftEqu = leftSide

    def solveEquation(self):
        self.convertEquationIntoGeneralForm()
        sympy_eq = sympify("Eq(" + self.equation.replace("=", ",") + ")")
        roots = solve(sympy_eq)

        return roots

def solve_equation(equation):
    solution = Solver(equation)
    roots = solution.solveEquation()
    return roots

#equation = input("Enter the equation: ")
#roots = solve_equation(equation)
#print("Roots:", roots)


In [ ]:
import cmath

class Solver:
    def __init__(self, equation):
        self.equation = equation

    def parse_equation(self):

        left, right = self.equation.split("=")

        parts = [part for part in (left + "-" + right).replace("-", "+-").split("+") if part]

        coefficients = {'x^2': 0, 'x': 0, 'const': 0}

        for part in parts:
            if 'x^2' in part:
                coefficients['x^2'] += float(part.replace('x^2', '') or '1')
            elif 'x' in part:
                coefficients['x'] += float(part.replace('x', '') or '1')
            else:
                coefficients['const'] += float(part)

        return coefficients

    def solve_equation(self):
        coefficients = self.parse_equation()
        a = coefficients['x^2']
        b = coefficients['x']
        c = coefficients['const']

        disc = b**2 - 4*a*c

        if a != 0:

            if disc < 0:
                return [
                    (-b + cmath.sqrt(disc)) / (2*a),
                    (-b - cmath.sqrt(disc)) / (2*a)
                ]

            else:
                return [
                    (-b + disc**0.5) / (2*a),
                    (-b - disc**0.5) / (2*a)
                ]
        elif b != 0:

            return [-c/b]
        else:

            if c == 0:
                return ['All numbers are solutions']
            else:
                return ['No solution']

    def solve_polynomial_equation(self):
      coefficients = self.parse_equation(self)
      degree = max(coefficient.keys(), key = lambda x: coefficients[x])

      if degree == 'x^2':
          return self.solve_equation()
      elif degree == 'x':
          return self.solve_linear_equation(coefficients)
      elif degree == 'const':
          return self.solve_constant_equation(coefficients)
      else:
          return ['No valid polynomial equation']


def solve_equation(equation):
    solution = Solver(equation)
    roots = solution.solve_equation()
    return roots

#equation = input("Enter the equation: ")
#roots = solve_equation(equation)
#print("Roots:", roots)

In [ ]:
def predictFromArray(arr):
    result = model.predict(arr)
    predicted_classes = np.argmax(result, axis=-1)
    return predicted_classes

In [ ]:
def solution():
    img = cv2.imread('/content/sample_data/unnamed.jpg', cv2.IMREAD_GRAYSCALE)
    img = ~img
    ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    ctrs, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    img_data = []
    rects = []
    for c in cnt:
        x, y, w, h = cv2.boundingRect(c)
        rect = [x, y, w, h]
        rects.append(rect)
    bool_rect = []
    for r in rects:
        l = []
        for rec in rects:
            flag = 0
            if rec != r:
                if r[0] < (rec[0] + rec[2] + 10) and rec[0] < (r[0] + r[2] + 10) and r[1] < (rec[1] + rec[3] + 10) and \
                        rec[1] < (r[1] + r[3] + 10):
                    flag = 1
                l.append(flag)
            if rec == r:
                l.append(0)
        bool_rect.append(l)
    dump_rect = []
    for i in range(0, len(cnt)):
        for j in range(0, len(cnt)):
            if bool_rect[i][j] == 1:
                area1 = rects[i][2] * rects[i][3]
                area2 = rects[j][2] * rects[j][3]
                if area1 == min(area1, area2):
                    dump_rect.append(rects[i])
    print('dump_rects:', dump_rect)
    final_rect = [i for i in rects if i not in dump_rect]
    print('final_rects:', final_rect)
    for r in final_rect:
        x = r[0]
        y = r[1]
        w = r[2]
        h = r[3]
        im_crop = thresh[y:y + h + 10, x:x + w + 10]

        im_resize = cv2.resize(im_crop, (28, 28))

        im_resize = np.reshape(im_resize, (1, 28, 28))
        img_data.append(im_resize)

    mainEquation = []
    operation = ''
    for i in range(len(img_data)):
        img_data[i] = np.array(img_data[i])
        img_data[i] = img_data[i].reshape(1, 28, 28, 1)
        result = predictFromArray(img_data[i])
        print(f"Predictions made: {len(mainEquation)}")
        if len(result) > 0:
            i = result[0]
            mainEquation.append(labels[i])

    StringEquation = ""
    for i in range(len(mainEquation)):
        a = mainEquation[i]
        if a.isdigit() == False and a.isalpha() == False and i < len(mainEquation) - 1:
            if a == mainEquation[i + 1] == '-':
                StringEquation += '='
            else:
                StringEquation += a
        if a.isalpha():
            if i > 0 and mainEquation[i - 1].isdigit():
                StringEquation += "*" + a
            else:
                StringEquation += a
        if a.isdigit():
            if i > 0:
                if mainEquation[i - 1].isdigit():
                    StringEquation += a
                elif mainEquation[i - 1].isalpha():
                    StringEquation += "^" + a
                else:
                    StringEquation += a
            else:
                StringEquation += a

    newStr = ""
    l = list(StringEquation)
    for i in range(len(l)):
        if l[i] == "=":
            newStr = l[:i + 1] + l[i + 2:]
    equ = ""
    for i in newStr:
        equ += i
    solution = Solver(equ)

    str1 = ''
    roots = solution.solve_equation()
    st = []
    for i in roots:
        i = str(i)
        st.append(i)

    str1 = ', '.join(st)

    solving(equ, str1)

In [ ]:
from tkinter import *

root = Tk()
root.resizable(0,0)
root.title('Equation Solver')

lasx,lasy=None,None

cv=Canvas(root,width=1200,height=500,bg='white')
#cv=Canvas(root,width=1600,height=500,bg='white')
cv.grid(row=0,column=0,pady=2,sticky=W,columnspan=2)
cve2=Label(root)
cve=Label(root,font=("Helvetica", 16))
cve.grid(row=0, column=1,pady=1, padx=1)
cve2.grid(row=1, column=1,pady=1, padx=1)

cv.bind('<Button-1>',activate_event)
cv.bind('<B1-Motion>',draw_smth)
btn_save=Button(text="Save",command=save,bg='#6495ED',fg='Black')
btn_save.grid(row=2,column=0,pady=1,padx=1)

btn_predict=Button(text="Predict",command=solution,bg='#6495ED',fg='Black')
btn_predict.grid(row=2,column=1,pady=1,padx=1)


def solving(equ,roots):

    cve2.configure(text='Your Equation is : '+equ)
    cve.configure(text='Result : '+(roots)+'\n')
root.mainloop()